In [ ]:
import kagglehub
import os
import random

random.seed(42)

directory_real = kagglehub.dataset_download("aibloy/fairface")
path_real = os.path.join(directory_real, "FairFace")

print("Path to dataset files:", directory_real)


Path to dataset files: /root/.cache/kagglehub/datasets/aibloy/fairface/versions/1


In [ ]:
import pandas as pd

image_path = "/root/.cache/kagglehub/datasets/aibloy/fairface/versions/1/FairFace"

train_fair = pd.DataFrame(pd.read_csv(f'{path_real}/train_labels.csv'))
train_fair['image_id'] = train_fair['file'].astype(str).apply(lambda x: x.split('/')[-1])
train_fair["gender"] = train_fair["gender"].replace({"Male": 1, "Female": 0}).astype(int)
train_fair["race"] = train_fair["race"].apply(lambda x: 1 if x == "White" else 0)
train_fair['label'] = 1
train_fair = train_fair[['image_id', 'file', 'age', 'race', 'gender', 'label']]
train_fair = train_fair.rename(columns={'file': 'image_path', 'race': 'ethnicity'})

train_fair.loc[:, 'image_path'] = train_fair['image_id'].apply(lambda x: os.path.join(image_path, "train", x))

val_fair = pd.DataFrame(pd.read_csv(f'{path_real}/val_labels.csv'))
val_fair['image_id'] = val_fair['file'].astype(str).apply(lambda x: x.split('/')[-1])
val_fair["gender"] = val_fair["gender"].replace({"Male": 1, "Female": 0}).astype(int)
val_fair["race"] = val_fair["race"].apply(lambda x: 1 if x == "White" else 0)
val_fair['label'] = 1
val_fair = val_fair[['image_id', 'file', 'age', 'race', 'gender', 'label']]
val_fair = val_fair.rename(columns={'file': 'image_path', 'race': 'ethnicity'})

val_fair.loc[:, 'image_path'] = val_fair['image_id'].apply(lambda x: os.path.join(image_path, "val", x))


<ipython-input-32-be5d6d346fb4>:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  train_fair["gender"] = train_fair["gender"].replace({"Male": 1, "Female": 0}).astype(int)
<ipython-input-32-be5d6d346fb4>:17: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  val_fair["gender"] = val_fair["gender"].replace({"Male": 1, "Female": 0}).astype(int)


In [ ]:
fair_face = pd.concat([train_fair, val_fair])

white = fair_face.where(fair_face['ethnicity'] == 1).dropna()
non_white = fair_face.where(fair_face['ethnicity'] == 0).dropna()

non_white_sample = non_white.sample(n=white.shape[0])

fair_face_balanced = pd.concat([white, non_white_sample])
fair_face_balanced

,image_id,image_path,age,ethnicity,gender,label
5,6.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,20-29,1.0,1.0,1.0
8,9.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,10-19,1.0,1.0,1.0
15,16.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,30-39,1.0,0.0,1.0
23,24.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,30-39,1.0,0.0,1.0
24,25.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,20-29,1.0,0.0,1.0
...,...,...,...,...,...,...
76636,76637.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,30-39,0.0,1.0,1.0
86644,86645.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,20-29,0.0,1.0,1.0
4524,4525.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,30-39,0.0,1.0,1.0
45769,45770.jpg,/root/.cache/kagglehub/datasets/aibloy/fairfac...,10-19,0.0,1.0,1.0
